# Chapter 6: Decision Trees

Decision Trees merupakan algoritma Machine Learning yang **sangat intuitif dan mudah dipahami**, namun tetap memiliki kemampuan untuk memodelkan hubungan yang kompleks.

Decision Tree dapat digunakan untuk:
- classification,
- regression,
- bahkan multi-output tasks.

Pada chapter ini, kita akan mempelajari bagaimana Decision Tree bekerja, bagaimana cara melatih dan memvisualisasikannya, serta memahami kekuatan dan keterbatasannya.

Decision Trees juga menjadi komponen dasar dari algoritma ensemble yang sangat populer seperti **Random Forest**, sehingga pemahaman chapter ini sangat penting.

## 1. Training and Visualizing a Decision Tree

Untuk memahami cara kerja Decision Tree, kita akan melatih sebuah model **DecisionTreeClassifier** menggunakan dataset Iris.

Dataset ini sering digunakan sebagai contoh karena sederhana, terstruktur dengan baik, dan mudah divisualisasikan.

### 1.1 Training a Decision Tree Classifier

Pada contoh ini, kita hanya menggunakan dua fitur:
- petal length
- petal width

Pembatasan jumlah fitur ini memudahkan kita untuk memahami bagaimana Decision Tree membuat keputusan.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:]  # petal length and petal width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

Model Decision Tree di atas dibatasi dengan `max_depth=2`, sehingga struktur pohon tetap sederhana dan mudah diinterpretasikan.

Tanpa pembatasan ini, Decision Tree cenderung tumbuh sangat dalam dan berisiko **overfitting**.

### 1.2 Visualizing the Decision Tree

Salah satu keunggulan utama Decision Tree adalah kemampuannya untuk divisualisasikan.

Visualisasi ini membantu kita memahami aturan keputusan yang dipelajari model.

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file="iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

File `.dot` yang dihasilkan dapat dikonversi menjadi gambar menggunakan **Graphviz**.

Visualisasi pohon ini menunjukkan bagaimana Decision Tree membagi ruang fitur berdasarkan threshold tertentu, seperti yang diilustrasikan pada gambar di buku.

## 2. How Decision Trees Make Predictions

Untuk memahami Decision Tree secara mendalam, kita perlu memahami **bagaimana model ini membuat prediksi**.

Decision Tree bekerja dengan cara menelusuri pohon dari **root node** hingga **leaf node**, berdasarkan aturan keputusan (*decision rules*) yang dipelajari selama training.

### 2.1 Traversing the Tree

Setiap node internal pada Decision Tree berisi sebuah pertanyaan berbentuk kondisi, misalnya:

> Apakah *petal length* ≤ 2.45?

Jika kondisi bernilai benar, instance akan diarahkan ke cabang kiri; jika salah, ke cabang kanan.

Proses ini berulang hingga mencapai leaf node, di mana prediksi akhir dibuat.

### 2.2 Node Information Explained

Setiap node pada visualisasi Decision Tree biasanya menampilkan beberapa informasi penting:

- **gini**: ukuran impurity (ketidakmurnian) pada node
- **samples**: jumlah instance data yang mencapai node tersebut
- **value**: distribusi jumlah instance pada setiap kelas
- **class**: kelas mayoritas pada node tersebut

Informasi ini membantu kita memahami keputusan yang dibuat oleh model.

### 2.3 Gini Impurity

**Gini impurity** mengukur seberapa sering sebuah instance akan salah diklasifikasikan jika kita memilih kelas secara acak berdasarkan distribusi kelas pada node tersebut.

Nilai Gini impurity:
- 0 → semua instance pada node berasal dari satu kelas (node murni)
- semakin besar → semakin bercampur kelasnya

Decision Tree berusaha meminimalkan Gini impurity pada setiap split.

### 2.4 Making a Prediction (Conceptual Example)

Misalkan sebuah bunga Iris memiliki:
- petal length = 5.0
- petal width = 1.5

Model akan mengevaluasi kondisi di root node terlebih dahulu, lalu mengikuti cabang yang sesuai hingga mencapai leaf node.

Kelas yang diprediksi adalah kelas mayoritas pada leaf node tersebut.

### 2.5 Predicting Class Probabilities

Selain memprediksi kelas, Decision Tree juga dapat menghasilkan **probabilitas kelas**.

Probabilitas ini dihitung berdasarkan proporsi kelas pada leaf node tempat instance berakhir.

In [ ]:
tree_clf.predict_proba([[5, 1.5]])

Output di atas menunjukkan probabilitas setiap kelas Iris.

Kelas dengan probabilitas tertinggi akan dipilih sebagai hasil prediksi akhir.

## 3. Estimating Class Probabilities

Selain menghasilkan prediksi kelas, Decision Tree juga mampu memperkirakan **probabilitas setiap kelas**.

Probabilitas ini dihitung berdasarkan distribusi kelas pada **leaf node** tempat sebuah instance berakhir.

Pendekatan ini membuat Decision Tree cukup informatif, karena kita tidak hanya mengetahui hasil prediksi, tetapi juga tingkat keyakinan model terhadap prediksi tersebut.

Secara intuitif:
- jika sebuah leaf node berisi instance dari satu kelas saja, probabilitas kelas tersebut adalah 100%
- jika leaf node berisi campuran beberapa kelas, probabilitas dihitung sebagai proporsi masing-masing kelas

Probabilitas ini sering digunakan dalam sistem yang membutuhkan **confidence score**.

## 4. The CART Training Algorithm

Decision Tree pada Scikit-Learn menggunakan algoritma **CART (Classification and Regression Trees)**.

CART bekerja dengan mencari split terbaik pada setiap node yang menghasilkan pemisahan data paling murni berdasarkan kriteria tertentu.

### 4.1 Gini Impurity

Pada classification, kriteria default yang digunakan oleh CART adalah **Gini impurity**.

Gini impurity mengukur seberapa sering sebuah instance akan salah diklasifikasikan jika kita secara acak memberikan label berdasarkan distribusi kelas pada node tersebut.

Decision Tree memilih split yang menghasilkan **penurunan Gini impurity terbesar**.

### 4.2 Entropy (Information Gain)

Sebagai alternatif dari Gini impurity, Decision Tree juga dapat menggunakan **entropy** sebagai kriteria split.

Entropy berasal dari teori informasi dan mengukur tingkat ketidakpastian pada sebuah node.

Split terbaik adalah split yang memberikan **information gain** terbesar, yaitu penurunan entropy yang paling signifikan.

### 4.3 Gini vs Entropy

Secara praktik, perbedaan antara Gini dan entropy biasanya kecil.

- **Gini impurity** cenderung sedikit lebih cepat secara komputasi
- **Entropy** cenderung menghasilkan pohon yang sedikit lebih seimbang

Pada kebanyakan kasus, penggunaan Gini impurity sudah cukup dan menjadi pilihan default.

### 4.4 Using Entropy in Scikit-Learn

Scikit-Learn memungkinkan kita untuk memilih kriteria split dengan mengatur parameter `criterion`.

In [ ]:
tree_clf_entropy = DecisionTreeClassifier(criterion="entropy", max_depth=2)
tree_clf_entropy.fit(X, y)

Model di atas menggunakan entropy sebagai kriteria split.

Struktur pohon yang dihasilkan mungkin sedikit berbeda, tetapi performa umumnya mirip dengan model berbasis Gini.

## 5. Regularization Hyperparameters

Salah satu kelemahan utama Decision Tree adalah kecenderungannya untuk **overfitting**, terutama jika pohon dibiarkan tumbuh tanpa batas.

Untuk mengatasi hal ini, Decision Tree menyediakan berbagai **hyperparameter regularization** yang berfungsi membatasi kompleksitas model.

### 5.1 Maximum Depth (`max_depth`)

Parameter `max_depth` membatasi kedalaman maksimum pohon.

- Nilai kecil → pohon lebih sederhana → risiko underfitting
- Nilai besar → pohon lebih kompleks → risiko overfitting

Mengatur `max_depth` merupakan salah satu cara paling efektif untuk mengontrol kompleksitas Decision Tree.

### 5.2 Minimum Samples for a Split (`min_samples_split`)

Parameter `min_samples_split` menentukan jumlah minimum instance yang dibutuhkan agar sebuah node dapat di-split.

Nilai yang lebih besar akan mencegah pemisahan node yang hanya berisi sedikit data, sehingga mengurangi overfitting.

### 5.3 Minimum Samples per Leaf (`min_samples_leaf`)

`min_samples_leaf` menentukan jumlah minimum instance yang harus ada pada setiap leaf node.

Parameter ini sangat efektif untuk menghaluskan prediksi, karena mencegah leaf node yang hanya berisi satu atau dua instance.

### 5.4 Maximum Features (`max_features`)

Parameter `max_features` membatasi jumlah fitur yang dipertimbangkan saat mencari split terbaik.

Dengan membatasi fitur, model menjadi lebih sederhana dan kurang sensitif terhadap noise.

### 5.5 Example: Regularized Decision Tree

Berikut contoh Decision Tree dengan beberapa parameter regularization yang diaktifkan.

In [ ]:
tree_clf_reg = DecisionTreeClassifier(
    max_depth=3,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

tree_clf_reg.fit(X, y)

Dengan regularization, Decision Tree menjadi lebih stabil dan memiliki kemampuan generalisasi yang lebih baik pada data baru.

Pemilihan nilai hyperparameter biasanya dilakukan melalui cross-validation.

## 7. Instability of Decision Trees

Meskipun Decision Tree sangat intuitif dan mudah diinterpretasikan, model ini memiliki kelemahan penting, yaitu **sensitivitas yang tinggi terhadap perubahan kecil pada data**.

Perubahan kecil pada dataset (misalnya penambahan atau penghapusan beberapa instance) dapat menghasilkan struktur pohon yang sangat berbeda.

### 7.1 Sensitivity to Data Variations

Karena Decision Tree membuat keputusan berdasarkan split terbaik secara lokal, sedikit perubahan pada data dapat mengubah split awal.

Perubahan ini kemudian *berantai* ke seluruh struktur pohon, menghasilkan model yang sangat berbeda meskipun datanya hampir sama.

### 7.2 Sensitivity to Axis Orientation

Decision Tree juga sensitif terhadap orientasi sumbu fitur.

Jika data diputar (misalnya melalui transformasi linier), Decision Tree mungkin memerlukan lebih banyak split untuk memisahkan data dengan baik.

Hal ini terjadi karena Decision Tree membuat split **sejajar sumbu fitur** (*axis-aligned splits*).

Kelemahan ini menjadi motivasi utama pengembangan metode ensemble seperti **Random Forest**, yang menggabungkan banyak Decision Tree untuk mengurangi variance dan meningkatkan stabilitas.

## Closing Summary (Chapter 6)

Pada Chapter 6, kita mempelajari Decision Trees sebagai algoritma Machine Learning yang kuat dan mudah diinterpretasikan.

Konsep utama yang dibahas meliputi:
- cara melatih dan memvisualisasikan Decision Tree,
- mekanisme prediksi dan perhitungan probabilitas kelas,
- kriteria split seperti Gini impurity dan entropy,
- regularization untuk mencegah overfitting,
- penggunaan Decision Tree untuk regresi,
- serta keterbatasan Decision Tree terkait stabilitas.

Meskipun memiliki kelemahan, Decision Tree menjadi fondasi penting bagi algoritma ensemble modern.

Pemahaman yang baik terhadap Decision Tree mempersiapkan kita untuk mempelajari model yang lebih kuat dan stabil pada chapter berikutnya, seperti **Ensemble Learning dan Random Forests**.